# Feature Selection
  
- Decision tree 모델을 활용해 feature selection 성능 확인  
참고 주소: https://www.geeksforgeeks.org/feature-selection-using-decision-tree/  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Commit_test_folder/LGAimers-06-2/src

/content/drive/MyDrive/Commit_test_folder/LGAimers-06-2/src


In [3]:
import pandas as pd
import numpy as np
import lgAimersDPP as dpp

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

In [4]:
train = pd.read_csv('data/train.csv')
train = dpp.dpp(train)
print(train.shape)
train.head()

(256351, 80)


,시술 당시 나이,시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,여성 주 불임 원인,여성 부 불임 원인,부부 주 불임 원인,부부 부 불임 원인,불명확 불임 원인,불임 원인 - 난관 질환,불임 원인 - 남성 요인,불임 원인 - 배란 장애,불임 원인 - 여성 요인,불임 원인 - 자궁경부 문제,불임 원인 - 자궁내막증,불임 원인 - 정자 농도,불임 원인 - 정자 면역학적 요인,불임 원인 - 정자 운동성,불임 원인 - 정자 형태,총 시술 횟수,클리닉 내 총 시술 횟수,IVF 시술 횟수,DI 시술 횟수,총 임신 횟수,IVF 임신 횟수,DI 임신 횟수,총 출산 횟수,IVF 출산 횟수,DI 출산 횟수,총 생성 배아 수,미세주입된 난자 수,미세주입에서 생성된 배아 수,이식된 배아 수,미세주입 배아 이식 수,저장된 배아 수,미세주입 후 저장된 배아 수,해동된 배아 수,해동 난자 수,수집된 신선 난자 수,저장된 신선 난자 수,혼합된 난자 수,파트너 정자와 혼합된 난자 수,기증자 정자와 혼합된 난자 수,난자 출처,정자 출처,동결 배아 사용 여부,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부,임신 성공 여부,난자 나이,정자 나이,isIVF,isICSI,isIUI,isICI,isGIFT,isFER,isGenericDI,isIVI,isBLASTOCYST,isAH,isUnknown,기증용 배아,난자 저장용 배아,배아 저장용 배아,연구용 배아,현재 시술용 배아,난자 채취 여부,난자 해동 여부,난자 혼합 여부,배아 이식 여부,배아 해동 여부
0,1,1,1,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,5.0,4.0,2.0,2.0,2.0,2.0,0.0,0.0,7.0,0.0,5.0,5.0,0.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0
1,6,1,0,-1,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0,6,6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0
2,1,1,1,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,7.0,7.0,0.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0
3,2,1,1,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,4.0,0.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0,2,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0
4,1,1,1,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,6.0,6.0,2.0,2.0,0.0,0.0,0.0,0.0,7.0,0.0,6.0,6.0,0.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0


In [5]:
X = train.drop('임신 성공 여부', axis=1)
Y = train['임신 성공 여부']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [34]:
# Train a Decision Tree Classifier
#featue selection을 수행하지 않은 모델.
clf = DecisionTreeClassifier(max_depth=20, random_state=8)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)

In [35]:
#feature selection과 성능 비교.
# Get feature importances
importances = clf.feature_importances_

# Select features with importance greater than a threshold
threshold = 0.005  # Adjust as needed
selected_features = X.columns[importances > threshold]

# Use only the selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

In [36]:
# Train a new model using the selected features
clf_selected = DecisionTreeClassifier(max_depth=20, random_state=8)
clf_selected.fit(X_train_selected, Y_train)

DecisionTreeClassifier(max_depth=20, random_state=8)

In [37]:
# Make predictions on the test set using the model trained with all features
y_pred_all_features = clf.predict(X_test)

# Calculate the accuracy of the model with all features
accuracy_all_features = accuracy_score(Y_test, y_pred_all_features)
print(f"Accuracy with all features: {accuracy_all_features}")

# Make predictions on the test set using the model trained with selected features
y_pred_selected_features = clf_selected.predict(X_test_selected)

# Calculate the accuracy of the model with selected features
accuracy_selected_features = accuracy_score(Y_test, y_pred_selected_features)
print(f"Accuracy with selected features: {accuracy_selected_features}")

Accuracy with all features: 0.7192564997757017
Accuracy with selected features: 0.7174426088822141


In [38]:
test = pd.read_csv('data/test.csv')
#test 데이터 전처리
test = dpp.dpp(test)

In [40]:
#feature selection과 성능 비교.
# Get feature importances
importances = clf.feature_importances_

# Select features with importance greater than a threshold
threshold = 0.005  # Adjust as needed
selected_features = test.columns[importances > threshold]

# Use only the selected features
test_selected = test[selected_features]

In [41]:
print(test_selected.shape)
print(X_test_selected.shape)

(90067, 31)
(51271, 31)


In [17]:
pred_proba = clf_selected.predict(test_selected)

res = pd.read_csv('data/sample_submission.csv')
res['probability'] = pred_proba

In [19]:
res.to_csv('data/featureSelection_test.csv', index=False)